In [2]:
import math
from random import random
import numpy as np

In [3]:
def print_mean_var(seq, mean, var, x, y):
    print("mean real: " + str(np.array(seq).mean()))
    print("mean theory: " + str(mean(x, y)))
    print("mean difference: " + str(np.array(seq).mean() - mean(x, y)))
    print()
    print("variance real: " + str(np.array(seq).var()))
    print("variance theory: " + str(var(x, y)))
    print("variance difference: " + str(np.array(seq).var() - var(x, y)))



def get_next_gauss(N):
    r = random()
    return math.sqrt(12 / N) * (sum([random() for _ in range(N)]) - N / 2)


def gauss(m, s, N, n):
    for _ in range(n):
        yield m + s * get_next_gauss(N)
    

def mean_gauss_theory(m, s):
    return m


def variance_gauss_theory(m, s):
    return s ** 2

def variance(seq):
    sum = 0
    mean = np.array(seq).mean()

    for el in seq:
        sum += (el - mean) ** 2
    return  1 / (len(seq) - 1) * sum

In [4]:
N = 48
n = 10000
m = -4
s = 2

gauss_seq = list(gauss(m, s, N, n))
print_mean_var(gauss_seq, mean_gauss_theory, variance_gauss_theory, m, s)


mean real: -3.9698743423858827
mean theory: -4
mean difference: 0.03012565761411734

variance real: 4.012996456992167
variance theory: 4
variance difference: 0.012996456992167005


In [5]:
a = 0.5

def mean_exp_theory(a, *args):
    return 1.0 / a

def variance_exp_theory(a, *args):
    return 1 / (a ** 2)

def exponential(a, n):
    for _ in range(n):
        yield -1 / a * math.log(random())
        
exp_seq = list(exponential(a, n))
print((np.array(exp_seq)).mean())
print_mean_var(exp_seq, mean_exp_theory, variance_exp_theory, a, None)

2.0160983248879076
mean real: 2.0160983248879076
mean theory: 2.0
mean difference: 0.016098324887907633

variance real: 3.948306141514765
variance theory: 4.0
variance difference: -0.05169385848523511


In [6]:
a = 0
b = 1.5

def mean_log_theory(a, b):
    return a

def variance_log_theory(a, b):
    return (math.pi ** 2) / 3 * b ** 2

def logistic(a, b, n):
    for _ in range(n):
        r = random()
        yield a + b * math.log(r  /(1 - r))
        
log_seq = list(logistic(a, b, n))
print_mean_var(log_seq, mean_log_theory, variance_log_theory, a, b)

mean real: -0.022063682190827973
mean theory: 0
mean difference: -0.022063682190827973

variance real: 7.36641009002865
variance theory: 7.4022033008170185
variance difference: -0.03579321078836806


In [7]:
pi = 0.7

3) (1 балл) Осуществить моделирование n = 10000 реализаций случайной величины из стандартного нормального закона распределения N(0, 1), используя преобразование Бокса — Мюллера http://ru.wikipedia.org/wiki/Преобразование_Бокса_—_Мюллера (в источнике  Лобач В.И. [и др] такой метод моделирования называется: моделирование, «используя функциональное преобразование»).


In [8]:
def box_muller(n):
    for i in range(n):
        x = random() * 2 - 1
        y = random() * 2 - 1
        s = x ** 2 + y ** 2
    
        while s > 1 or s == 0:
            x = random() * 2 - 1
            y = random() * 2 - 1
            s = x ** 2 + y ** 2
    
        yield x *( -2 * math.log(s) / s) ** 0.5
        yield y *( -2 * math.log(s) / s) ** 0.5

bm_seq = list(box_muller(n))
for i in range(20):
    print(bm_seq[i])

-0.09332064772319325
-0.10566579403894813
0.44331278706193966
0.20092140680763157
-1.045097083949485
-0.7204304856784868
1.7341604486306037
-0.26177681970449024
1.4713249565960116
-0.6129352249998252
-0.25386552024501363
-0.4667582127600374
0.39111549762933884
-1.43573906594666
0.45182883732187185
-1.6159264800582902
0.13605874841793286
-1.0234692563881274
-0.9589613902754903
-0.6123488337260174


4) (1 балл) для смоделированной в бонусном пункте 3 выборки оценить коэффициент корреляции между элементами, стоящими на четных позициях, и элементах, стоящих на нечетных позициях.

In [9]:
odd_elements = [bm_seq[i] for i in range(0, n, 2)]
even_elements = [bm_seq[i] for i in range(1, n, 2)]

def correlation(seq1, seq2):
    mean1 = np.array(seq1).mean()
    mean2 = np.array(seq2).mean()
    var1 = variance(seq1)
    var2 = variance(seq2)
    
    corr = 0.
    for i in range(len(seq1)):
        corr += (seq1[i] - mean1) * (seq2[i] - mean2)

    return corr / (var1 * var2) ** 0.5 / len(seq1)
correlation(odd_elements, even_elements)

-0.022544798953924583

In [67]:
e = 0.05
k = 10
delta_pearson = 16.9
def pearson(f, n, probabilities):
    xi = 0
    for i in range(len(f)):
        xi += (f[i] - probabilities[i] * n) ** 2 / (probabilities[i] * n)
    
    return xi

def count_frequencies(n, seq, probability_function):
    seq = sorted(seq, reverse=True)
    print(seq[:20])
    size = len(seq)
    k = int(size / n)
    probabilities = []
    frequencies = []
    j = 0
    for i in range(n):
        frequence = 0
        left = probability_function(min(seq) + (max(seq) - min(seq))* i / n)
        right = probability_function(min(seq) + (max(seq) - min(seq))* (i+1) / n)
        print("left: ", j)
        print("right border: ", min(seq) + (max(seq) - min(seq))* (i+1) / n)
        print("seq j: ", seq[j])

        while j < size and  seq[j] < min(seq) + (max(seq) - min(seq))* (i+1) / n:
            j += 1
            frequence += 1
        frequencies.append(frequence)
        print("right: ", j)
        probabilities.append(right - left)
    return frequencies, k, probabilities




In [70]:
print(max(gauss_seq))
print(min(gauss_seq))


def gauss_distribution(x):
    return 0.5 * (1 + math.erf((x - m) / (math.sqrt(2) * s)))

f, segment_size, probabilities = count_frequencies(k, gauss_seq, gauss_distribution)
print(probabilities)
print("FREQUENCES")
print(f)

pearson_gauss = pearson(f, len(gauss_seq), probabilities)
print("Pearson gauss: ", pearson_gauss, "< ", delta_pearson, "? ", pearson_gauss < delta_pearson)

4.596339015329512
-11.180597574979213
[4.596339015329512, 3.3062942166077285, 3.145478412160447, 2.8731299981985288, 2.7014127573376285, 2.6324376967877576, 2.2360464932320347, 2.160690026195301, 2.1561968151035913, 2.0720711975965855, 1.9842822572907686, 1.978432366414264, 1.9202535625762707, 1.912011701366776, 1.8566660525199445, 1.8261349044147117, 1.7011161764843692, 1.6842255366276255, 1.6280501194031949, 1.6210158577664018]
left:  0
right border:  -9.60290391594834
seq j:  4.596339015329512
right:  0
left:  0
right border:  -8.025210256917468
seq j:  4.596339015329512
right:  0
left:  0
right border:  -6.447516597886596
seq j:  4.596339015329512
right:  0
left:  0
right border:  -4.869822938855723
seq j:  4.596339015329512
right:  0
left:  0
right border:  -3.2921292798248505
seq j:  4.596339015329512
right:  0
left:  0
right border:  -1.714435620793978
seq j:  4.596339015329512
right:  0
left:  0
right border:  -0.13674196176310538
seq j:  4.596339015329512
right:  0
left:  0
ri

In [69]:
m = -4
s = 2
def logistic_distribution(x):
    return 1 / (1 + math.exp(-1 * (x - m) / s))

f, segment_size, probabilities = count_frequencies(k, log_seq, logistic_distribution)

pearson_logistic = pearson(f, len(log_seq), probabilities)
print("Pearson logistic: ", pearson_logistic, "< ", delta_pearson, "? ", pearson_logistic < delta_pearson)

[13.903617750819038, 12.012453647205831, 11.857340105720247, 11.793634192419558, 11.492433183699003, 11.103973601183508, 10.965997024688006, 10.879141115013466, 10.058950761962778, 9.921231599829694, 9.873037534160769, 9.863702370790708, 9.73878032588226, 9.540724914283262, 9.496058143369963, 9.463778506711018, 9.310288678059312, 9.244682208678775, 8.963542981383375, 8.8384989533772]
left:  0
right border:  -10.761151255489763
seq j:  13.903617750819038
right:  0
left:  0
right border:  -8.020621365899895
seq j:  13.903617750819038
right:  0
left:  0
right border:  -5.28009147631003
seq j:  13.903617750819038
right:  0
left:  0
right border:  -2.539561586720163
seq j:  13.903617750819038
right:  0
left:  0
right border:  0.2009683028697058
seq j:  13.903617750819038
right:  0
left:  0
right border:  2.9414981924595693
seq j:  13.903617750819038
right:  0
left:  0
right border:  5.682028082049435
seq j:  13.903617750819038
right:  0
left:  0
right border:  8.422557971639304
seq j:  13.9

In [45]:
from numpy import log
a_log = 2
b_log = 3
def logistic(a, b, n):
    for i in range(n):
        x = random()
        yield a + b * log(x / (1 - x))
logistic_seq = list(logistic(a_log, b_log, n))

f, segment_size, probabilities = count_frequencies(k, logistic_seq, logistic_distribution)

pearson_logistic = pearson(f, len(logistic_seq), probabilities)
print("Pearson logistic: ", pearson_logistic, "< ", delta_pearson, "? ", pearson_logistic < delta_pearson)

[25.11903094320389, 23.821637612385675, 23.758170935742612, 23.26995532348087, 23.002072379590892, 22.295105149019317, 22.092048502845145, 22.048561493887178, 21.47173339883626, 21.308481469847127]


ZeroDivisionError: float division by zero